# Individual flight approach

In [12]:
import requests

headers = {
    'authority': 'be.wizzair.com',
    'accept': 'application/json, text/plain, */*',
    'origin': 'https://wizzair.com',
    'x-requestverificationtoken': '07cfb2f963a84ac4b2a6924209304d77',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'content-type': 'application/json;charset=UTF-8',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'referer': 'https://wizzair.com/',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-GB,en;q=0.9,hu-HU;q=0.8,hu;q=0.7,en-US;q=0.6',
    'cookie': 'bm_sz=6A020030C69891810E40357F2D4DE30D~YAAQ3rL3SIq5TLxuAQAAXxCdwAWbC49MUl++tNzOruhbTkxq1QRwqVXpQH/PdddIjtCYgHurnLedF13He7iTNP4DE0aljJ8DPjAxNE+T1MXsw0leqkuvRuY1Gcu8QZImevpMeSofTVm4//aXwJUP96LExLnixraEZsAkjeSs+OHM92bVp+tUUD4843BfVWM7; _abck=42083D8A10BFD737B73317108419016E~0~YAAQ3rL3SJu5TLxuAQAAcRWdwALAUwIhfsgVvJnB/FypE8iZe9BidFxZRHe73C3KgG3peNR6U1KYiYzi2WHDz0y0nsas5xa973/rD9jdlrWI3oLMg0oNNq0lX3hdBIHE6atBbH3blymFcFrvgd1qBTwleEl83kKBhpOS4IZ++jWSkBpdbiyPlZpzUq/fgbToIsO2bESL+M1HNgMVettkEUL6SVKPxJcTwpoJ2YILGMTd6sJrzfvpmmHtwz/A1ky1asX1znPbTVRGznvEOy2YEfNkfHrcnzFiFdAJlXYFUCLe1Ek+m9PbNc8TsGCdnuKthwiyLZSBIQw=~-1~-1~-1; ak_bmsc=60F561E97D872E51CD8AF15AFA0FE53548F7B2DE704E00002D7CE35D8FB03567~plM3DzKt4g/EWhamf33FrUb6vNfK1fXaDjxtjpjMV1lOCdusUDcEmXGm4hffkGIGyc2ZX/HTTBojpF3a+/kPYO/X8S8pDukZS6Q1xij6nKvTxCFjgfUrOXDqk7HKbRuq/RRo6RZvdOs6U5+kmuGt+JtrT+8UH4XKELZ6getR5nRURsldS7G3LCR2xi6D27uvYcBagfqL4hma4S99O6orKND5Kf+KOZZSiZHgrmIQ6b+GKRGDkenXgDLR770D3MXqFqBq5Y4MK/4SxRFvSB0wmi3u1kCCnMkY+ZrfTHpA4qSpc1CFsJleqRhfzuPwg0n8ZhosTdJlzehit7eSBp5Pg2mw==; _hjid=5d25b846-c829-4cff-be40-5781b49bae6d; _gcl_au=1.1.2126801188.1575189569; _ga=GA1.2.1370863027.1575189570; _gid=GA1.2.1791855577.1575189570; _fbp=fb.1.1575189571756.118279262; ASP.NET_SessionId=5xeahvphrruslcy1qzklv13l; RequestVerificationToken=07cfb2f963a84ac4b2a6924209304d77; _gat_gtag_UA_2629375_25=1; bm_sv=10D3F2F41A433EE60BE47880FB3019E1~WLUCRAlnF6eEBUPS1933XOMgUpd/sSyfYGvg2UL2ynNzv+45hf7HUCBBFOjyIgHwU7XNmbIf+EPOEents5lPtEau42cd+0vl0ui1UwhOtiseYKi3G/SZm58ccGwhwpyEES3XTEb+zxtHgnqMN7eIGuphXfeOTOUWqjKlytJNoPw=',
}

In [13]:
data = {"isFlightChange":False,
        "isSeniorOrStudent":False,
        "flightList":[{"departureStation":"BUD",
                       "arrivalStation":"CRL",
                       "departureDate":"2020-06-16"}],
        "adultCount":1,
        "childCount":0,
        "infantCount":0,
        "wdc":True}

In [14]:
base = datetime.today() + timedelta(days = 2)
date_list = [base + timedelta(days=x) for x in range(365)]

In [15]:
prices_dict = {}
arrival_stations = ["CRL", "EIN", "TLV", "LTN"]
df_list = []
for date in tqdm(date_list):
    data["flightList"][0]["departureDate"] = date.strftime("%Y-%m-%d")
    for arrival_station in arrival_stations:
        data["flightList"][0]["arrivalStation"] = arrival_station
        response = requests.post('https://be.wizzair.com/10.1.0/Api/search/search', headers=headers, data=json.dumps(data))
        try:
            resp_data = response.json()
            ## Szerintem itt az 1-2 és a 4-5 azok az árak, amik majd meg fognak jelenni, amikor elfogynak ezek.
            prices_dict["normal_price"]  = [resp_data["outboundFlights"][0]["fares"][3]["basePrice"]["amount"]]
            prices_dict["wdc_price"] = [resp_data["outboundFlights"][0]["fares"][0]["basePrice"]["amount"]]
        except:
            prices_dict["normal_price"] = np.nan
            prices_dict["wdc_price"] = np.nan
        prices_dict["Date"] = [date.strftime("%Y-%m-%d")]
        prices_dict["Departure"] = ["BUD"]
        prices_dict["Destination"] = [arrival_station]
        df_list.append(pd.DataFrame.from_dict(prices_dict))


In [16]:
final = pd.concat(df_list).reset_index(drop = True)

In [17]:
final

,normal_price,wdc_price,Date,Departure,Destination
0,17890.0,14590.0,2019-12-03,BUD,CRL
1,16190.0,12990.0,2019-12-03,BUD,EIN
2,12990.0,9690.0,2019-12-04,BUD,CRL
3,16190.0,12990.0,2019-12-04,BUD,EIN
4,22690.0,19490.0,2019-12-05,BUD,CRL
...,...,...,...,...,...
725,NaN,NaN,2020-11-29,BUD,EIN
726,NaN,NaN,2020-11-30,BUD,CRL
727,NaN,NaN,2020-11-30,BUD,EIN
728,NaN,NaN,2020-12-01,BUD,CRL


In [9]:
data["flightList"][0]["arrivalStation"] = "LTN"
data["flightList"][0]["departureDate"] = "2019-12-24"
response = requests.post('https://be.wizzair.com/10.1.0/Api/search/search', headers=headers, data=json.dumps(data))
resp_data = response.json()

{'isFlightChange': False,
 'isSeniorOrStudent': False,
 'flightList': [{'departureStation': 'BUD',
   'arrivalStation': 'CRL',
   'departureDate': '2019-12-03'}],
 'adultCount': 1,
 'childCount': 0,
 'infantCount': 0,
 'wdc': True}

# Timetable approach

In [3]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import json

In [4]:
headers = {
    'authority': 'be.wizzair.com',
    'accept': 'application/json, text/plain, */*',
    'origin': 'https://wizzair.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'content-type': 'application/json;charset=UTF-8',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'referer': 'https://wizzair.com/en-gb/flights/timetable',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-GB,en;q=0.9,hu-HU;q=0.8,hu;q=0.7,en-US;q=0.6'}

data = {"flightList":[{"departureStation":"BUD",
                       "arrivalStation":"",
                       "from":"",
                       "to":""},
                      {"departureStation":"",
                       "arrivalStation":"BUD",
                       "from":"",
                       "to":""}],"priceType":"","adultCount":1,"childCount":0,"infantCount":0}
##priceType = "wdc"
#response = requests.post('https://be.wizzair.com/10.1.0/Api/search/timetable', headers=headers, data=json.dumps(data))

In [5]:
def alter_price(price_type, flights):
    if price_type == "wdc":
        [flight.update({"priceType": "wdc"}) for flight in flights]
    else:
        [flight.update({"priceType": "regular"}) for flight in flights]
    return flights

In [6]:
destinations = ["CRL", 
                "EIN", 
                "TLV", 
                "LTN", 
                "TIA", 
                "GYD", 
                "BRU", 
                "SJJ",
                "BOJ",
                "SOF",
                "LCA",
                "BOD",
                "NCE",
               "KUT",
               "SXF",
               "DTM",
               "FRA",
               "HAJ",
               "ATH",
               "CFU",
               "HER",
               "RHO",
               "SKG",
               "ZTH",
               "KEF",
               "ETM",
               "AHO",
               "BRI",
               "BLQ",
               "CTA",
               "MXP",
               "NAP",
               "FCO",
               "TSE",
               "PRN",
               "MLA",
               "TGD",
               "SKP",
               "OSL",
               "WAW",
               "FAO",
               "LIS",
               "OPO",
               "TGM",
               "KZN",
               "VKO",
               "LED",
               "ALC",
               "BCN",
               "CDT",
               "IBZ",
               "MAD",
               "AGP",
               "PMI",
               "TFS",
               "GOT",
               "MMX",
               "NYO",
               "BSL",
               "HRK",
               "IEV",
               "LWO",
               "ODS",
               "OZH",
               "DWC",
               "BHX",
               "DSA",
               "EDI",
               "GLA",
               "LPL",
               "LGW"]

In [10]:
data_list = []
base = datetime.today()
for period in range(12):
    data["flightList"][0]["from"] = (base + timedelta(days = period * 42)).strftime("%Y-%m-%d")
    data["flightList"][1]["from"] = (base + timedelta(days = period * 42)).strftime("%Y-%m-%d")
    
    data["flightList"][0]["to"] = (base + timedelta(days = (period + 1) * 42)).strftime("%Y-%m-%d")
    data["flightList"][1]["to"] = (base + timedelta(days = (period + 1) * 42)).strftime("%Y-%m-%d")
    
    for price_type in ["regular", "wdc"]:
        data["priceType"] = price_type
        for destination in tqdm(destinations):
            data["flightList"][0]["arrivalStation"] = destination
            data["flightList"][1]["departureStation"] = destination
            
            response = requests.post('https://be.wizzair.com/10.1.0/Api/search/timetable', headers=headers, data=json.dumps(data))
            try:
                data_list.append(alter_price(price_type, response.json()["outboundFlights"]))
            except:
                print(data)
            try:
                data_list.append(alter_price(price_type, response.json()["returnFlights"]))
            except:
                print(data)

In [11]:
flat_list = [item for sublist in data_list for item in sublist]

In [12]:
df = pd.DataFrame(flat_list)
df

,departureStation,arrivalStation,departureDate,price,priceType,departureDates,classOfService,hasMacFlight
0,BUD,CRL,2019-12-01T00:00:00,"{'amount': 45490.0, 'currencyCode': 'HUF'}",regular,[2019-12-01T18:05:00],GF,False
1,BUD,CRL,2019-12-02T00:00:00,"{'amount': 29190.0, 'currencyCode': 'HUF'}",regular,[2019-12-02T18:05:00],I,False
2,BUD,CRL,2019-12-03T00:00:00,"{'amount': 17890.0, 'currencyCode': 'HUF'}",regular,[2019-12-03T18:35:00],KL,False
3,BUD,CRL,2019-12-04T00:00:00,"{'amount': 12990.0, 'currencyCode': 'HUF'}",regular,[2019-12-04T06:15:00],B,False
4,BUD,CRL,2019-12-05T00:00:00,"{'amount': 22690.0, 'currencyCode': 'HUF'}",regular,[2019-12-05T18:20:00],J,False
...,...,...,...,...,...,...,...,...
44257,LTN,BUD,2020-10-20T00:00:00,"{'amount': 25990.0, 'currencyCode': 'HUF'}",wdc,"[2020-10-20T08:40:00, 2020-10-20T10:50:00, 202...",KW,True
44258,LTN,BUD,2020-10-21T00:00:00,"{'amount': 19490.0, 'currencyCode': 'HUF'}",wdc,"[2020-10-21T08:40:00, 2020-10-21T15:25:00, 202...",LW,True
44259,LTN,BUD,2020-10-22T00:00:00,"{'amount': 19490.0, 'currencyCode': 'HUF'}",wdc,"[2020-10-22T08:40:00, 2020-10-22T15:25:00, 202...",LW,True
44260,LTN,BUD,2020-10-23T00:00:00,"{'amount': 19490.0, 'currencyCode': 'HUF'}",wdc,"[2020-10-23T08:40:00, 2020-10-23T10:50:00, 202...",LW,True


In [18]:
len(df)

44262

In [15]:
cond_1 = df["departureDate"] == "2020-03-01T00:00:00"
cond_2 = df["priceType"] == "regular"
cond_3 = df["departureStation"] == "TLV"

In [17]:
len(df[cond_2])

22131

In [19]:
df.to_pickle("./wizz_19_12_01.pkl")